In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time

C:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from numpy.core.fromnumeric import shape
def load_data(path,batch_size,input_size):
    
    normalize = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]) 
    transform_dict = {"src":  normalize}  
    # train_path=path+"/train"
    # test_path=path+"/test"
    data = datasets.ImageFolder(root=path,transform=transform_dict["src"])
    # transform_dict1 = {"test":  normalize} 
    # test1 = datasets.ImageFolder(root=path,transform=transform_dict["test"])
    # transform_dict = {"test":  normalize}
    # test= datasets.ImageFolder(root=path,transform=transform_dict["test"])
    # train_size = int((1- (test_split + val_split)) * len(data))
    # test_size = int((1 - (val_split)) * len(data)) - train_size
    # val_size = len(data) - train_size - test_size
    train_size=int(0.75*len(data))
    print(len(data))
    test_size=int(len(data)-train_size)
    train, test = td.random_split(data,[train_size,test_size])

    data_loader_train = td.DataLoader(train,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    data_loader_test = td.DataLoader(test,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    # data_loader_val = td.DataLoader(val,batch_size=batch_size,shuffle=True,drop_last=False,num_workers=0)
    return data_loader_train, data_loader_test

In [3]:
data_loader_train,data_loader_test=load_data(r"C:\Users\Admin\Desktop\COMP 6721\Project\datasets\Dataset1",32,64)

21165


In [4]:
data_loader_train,data_loader_test=load_data(r"/content/drive/MyDrive/Project/datasets/Dataset2",32,40)

6922


In [5]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 4):
        super().__init__()
#         _log_api_usage_once(self)
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(64, 192, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(4096, num_classes),
        )

    def forward(self, x) :
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
import torch.optim as optim
net = AlexNet(3).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [7]:
###### Define and run your training loop here #########
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
net.to(device)

num_epochs = 20
total_steps = len(data_loader_train)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(data_loader_train):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
#         print("kjnfjnrnkrn",i)\\
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print("second time",i)
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

Device: cuda:0
Epoch [1/20], Step [10/163], Loss: 1.0981, Accuracy: 40.62%
Epoch [1/20], Step [20/163], Loss: 1.0981, Accuracy: 25.00%
Epoch [1/20], Step [30/163], Loss: 1.0626, Accuracy: 50.00%
Epoch [1/20], Step [40/163], Loss: 1.0912, Accuracy: 34.38%
Epoch [1/20], Step [50/163], Loss: 1.0934, Accuracy: 37.50%
Epoch [1/20], Step [60/163], Loss: 1.1392, Accuracy: 18.75%
Epoch [1/20], Step [70/163], Loss: 1.0302, Accuracy: 31.25%
Epoch [1/20], Step [80/163], Loss: 1.1423, Accuracy: 31.25%
Epoch [1/20], Step [90/163], Loss: 1.0073, Accuracy: 50.00%
Epoch [1/20], Step [100/163], Loss: 1.1445, Accuracy: 28.12%
Epoch [1/20], Step [110/163], Loss: 1.0799, Accuracy: 40.62%
Epoch [1/20], Step [120/163], Loss: 1.1202, Accuracy: 43.75%
Epoch [1/20], Step [130/163], Loss: 1.0691, Accuracy: 40.62%
Epoch [1/20], Step [140/163], Loss: 1.0952, Accuracy: 46.88%
Epoch [1/20], Step [150/163], Loss: 1.0431, Accuracy: 62.50%
Epoch [1/20], Step [160/163], Loss: 1.1520, Accuracy: 40.62%
Epoch [2/20], Step

In [8]:
######## Write your code here #############
net.eval() 
count=0
with torch.no_grad(): 
    correct = 0
    total = 0
    for data in data_loader_test:
        # count+=1
        # print(count)
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))

Test Accuracy of the model on the 1731 test images: 33.10225303292894 %


# Training the dataset with AlexNet fetching directly from Pytorch

In [4]:
import torch.optim as optim

import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

Using cache found in C:\Users\Admin/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Admin\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
###### Define and run your training loop here #########
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
model.to(device)

num_epochs = 5
total_steps = len(data_loader_train)
t1 = time.time()
for epoch in range(num_epochs):
    for i, data in enumerate(data_loader_train):
        images, labels = data[0].to(device), data[1].to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
#         print("kjnfjnrnkrn",i)\\
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print("second time",i)
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            
print("######## Training Finished in {} seconds ###########".format(time.time()-t1))


Device: cuda:0
Epoch [1/5], Step [10/497], Loss: 6.7373, Accuracy: 50.00%
Epoch [1/5], Step [20/497], Loss: 4.8142, Accuracy: 56.25%
Epoch [1/5], Step [30/497], Loss: 5.1484, Accuracy: 43.75%
Epoch [1/5], Step [40/497], Loss: 1.7266, Accuracy: 18.75%
Epoch [1/5], Step [50/497], Loss: 1.3228, Accuracy: 46.88%
Epoch [1/5], Step [60/497], Loss: 1.2216, Accuracy: 50.00%
Epoch [1/5], Step [70/497], Loss: 0.9579, Accuracy: 68.75%
Epoch [1/5], Step [80/497], Loss: 1.1964, Accuracy: 59.38%
Epoch [1/5], Step [90/497], Loss: 1.3997, Accuracy: 40.62%
Epoch [1/5], Step [100/497], Loss: 1.3866, Accuracy: 46.88%
Epoch [1/5], Step [110/497], Loss: 1.0768, Accuracy: 53.12%
Epoch [1/5], Step [120/497], Loss: 1.2694, Accuracy: 21.88%
Epoch [1/5], Step [130/497], Loss: 1.1974, Accuracy: 46.88%
Epoch [1/5], Step [140/497], Loss: 1.4556, Accuracy: 37.50%
Epoch [1/5], Step [150/497], Loss: 1.2073, Accuracy: 46.88%
Epoch [1/5], Step [160/497], Loss: 1.2672, Accuracy: 46.88%
Epoch [1/5], Step [170/497], Loss:

Epoch [3/5], Step [400/497], Loss: 0.5106, Accuracy: 90.62%
Epoch [3/5], Step [410/497], Loss: 0.6062, Accuracy: 75.00%
Epoch [3/5], Step [420/497], Loss: 0.3001, Accuracy: 87.50%
Epoch [3/5], Step [430/497], Loss: 0.5465, Accuracy: 84.38%
Epoch [3/5], Step [440/497], Loss: 0.6106, Accuracy: 71.88%
Epoch [3/5], Step [450/497], Loss: 0.4476, Accuracy: 81.25%
Epoch [3/5], Step [460/497], Loss: 0.4473, Accuracy: 75.00%
Epoch [3/5], Step [470/497], Loss: 0.5653, Accuracy: 78.12%
Epoch [3/5], Step [480/497], Loss: 0.5973, Accuracy: 75.00%
Epoch [3/5], Step [490/497], Loss: 0.4427, Accuracy: 87.50%
Epoch [4/5], Step [10/497], Loss: 0.9111, Accuracy: 68.75%
Epoch [4/5], Step [20/497], Loss: 0.7388, Accuracy: 71.88%
Epoch [4/5], Step [30/497], Loss: 0.5872, Accuracy: 78.12%
Epoch [4/5], Step [40/497], Loss: 0.9233, Accuracy: 56.25%
Epoch [4/5], Step [50/497], Loss: 0.6773, Accuracy: 78.12%
Epoch [4/5], Step [60/497], Loss: 1.0131, Accuracy: 46.88%
Epoch [4/5], Step [70/497], Loss: 0.7752, Accu

In [6]:
model.eval() 
perds = []
target = []
with torch.no_grad(): 
    correct = 0
    total = 0
    for data in data_loader_test:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        perds.extend(predicted)
        target.extend(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the {} test images: {} %'
        .format(total, (correct / total) * 100))
    
# plt.plot(Accuracies)

Test Accuracy of the model on the 5292 test images: 79.59183673469387 %


In [7]:
import torchmetrics

In [8]:
from torchmetrics.functional import precision_recall
from torchmetrics import F1Score
from torchmetrics import ConfusionMatrix
perds1 = torch.stack(perds)
target1 = torch.stack(target)
f1 = F1Score(num_classes=4).to(device)
print(f1(perds1, target1))
print(precision_recall(perds1, target1, average='macro', num_classes=4))
confmat = ConfusionMatrix(num_classes=4).to(device)
confmat(perds1, target1)

tensor(0.7959, device='cuda:0')
(tensor(0.8557, device='cuda:0'), tensor(0.7378, device='cuda:0'))


tensor([[ 458,  102,  375,    3],
        [  42, 1086,  380,    3],
        [  27,   74, 2408,    3],
        [   9,   12,   50,  260]], device='cuda:0')